In [1]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
from tqdm import tqdm

import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch

import matplotlib.pyplot as plt
import datetime
import pickle
import gzip

np.random.seed(datetime.datetime.now().microsecond)

PEAK_THRESHOLD = 390
DATA_PATH = "./data/exported/30sec/"

# Data Load

## Sitting

In [ ]:
# Sitting data load
with gzip.open(DATA_PATH + "sitting_ecg.pkl", "rb") as f:
    sitting_ecg = pickle.load(f)

with gzip.open(DATA_PATH + "sitting_acc.pkl", "rb") as f:
    sitting_acc = pickle.load(f)
sitting_acc = sitting_acc - np.mean(sitting_acc)

In [ ]:
fig, ax = plt.subplots(len(sitting_acc), 1, figsize=(20, 10))
for idx in range(len(sitting_acc)):
    peaks, _ = find_peaks(sitting_acc[idx], height=PEAK_THRESHOLD)
    np.diff(peaks)

    ax[idx].plot(sitting_acc[idx])
    ax[idx].plot(peaks, sitting_acc[idx][peaks], "x")
plt.show()

In [ ]:
peak_list = list()
for idx in range(len(sitting_acc)):
    peaks, _ = find_peaks(sitting_acc[idx], height=PEAK_THRESHOLD)
    if peaks == []:
        peak_list.append(np.array([0]))
    peak_list.append(len(peaks))

X_sitting = sitting_ecg
y_sitting = peak_list

## Walking

In [ ]:
# Walking data load
with gzip.open(DATA_PATH + "walking_ecg.pkl", "rb") as f:
    walking_ecg = pickle.load(f)

with gzip.open(DATA_PATH + "walking_acc.pkl", "rb") as f:
    walking_acc = pickle.load(f)
walking_acc = walking_acc - np.mean(walking_acc)

In [ ]:
fig, ax = plt.subplots(len(walking_acc), 1, figsize=(20, 20))
for idx in range(len(walking_acc)):
    peaks, _ = find_peaks(walking_acc[idx], height=PEAK_THRESHOLD)
    np.diff(peaks)

    ax[idx].plot(walking_acc[idx])
    ax[idx].plot(peaks, walking_acc[idx][peaks], "x")
plt.show()

In [ ]:
peak_list = list()
for idx in range(len(walking_acc)):
    peaks, _ = find_peaks(walking_acc[idx], height=PEAK_THRESHOLD)
    if peaks == []:
        peak_list.append(np.array([0]))
    peak_list.append(len(peaks))

X_walking = walking_ecg
y_walking = peak_list

In [ ]:
print(y_walking)

## Running

In [ ]:
# Running data load
with gzip.open(DATA_PATH + "running_ecg.pkl", "rb") as f:
    running_ecg = pickle.load(f)

with gzip.open(DATA_PATH + "running_acc.pkl", "rb") as f:
    running_acc = pickle.load(f)
running_acc = running_acc - np.mean(running_acc)

In [ ]:
fig, ax = plt.subplots(len(running_acc), 1, figsize=(20, 10))
for idx in range(len(running_acc)):
    peaks, _ = find_peaks(running_acc[idx], height=PEAK_THRESHOLD)
    np.diff(peaks)

    ax[idx].plot(running_acc[idx])
    ax[idx].plot(peaks, running_acc[idx][peaks], "x")
plt.show()

In [ ]:
peak_list = list()
for idx in range(len(running_acc)):
    peaks, _ = find_peaks(running_acc[idx], height=PEAK_THRESHOLD)
    if peaks == []:
        peak_list.append(np.array([0]))
    peak_list.append(len(peaks))

X_running = running_ecg
y_running = peak_list

# X, y data Split

In [ ]:
BATCH_SIZE = 1

In [ ]:
X = np.concatenate((X_walking, X_running))
y = np.concatenate((y_walking, y_running))

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# y = scaler.fit_transform(y.reshape(-1, 1))

print(f"""X shape: {X.shape}
y shape: {y.shape}""")

In [ ]:
plotting_idx = np.random.randint(0, len(X), 3)

# Plotting the data
fig, ax = plt.subplots(3, 1, figsize=(20, 10))

for idx in range(3):
    ax[idx].plot(X[plotting_idx[idx]])
    ax[idx].set_title(f"Peak count: {y[plotting_idx[idx]]}")
plt.show()


In [ ]:
with gzip.open('./data//exported/X.pkl', 'wb') as f:
    pickle.dump(X, f)

with gzip.open('./data//exported/y.pkl', 'wb') as f:
    pickle.dump(y, f)

In [ ]:
with gzip.open('./data//exported/X.pkl', 'rb') as f:
    X = pickle.load(f)

with gzip.open('./data//exported/y.pkl', 'rb') as f:
    y = pickle.load(f)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=datetime.datetime.now().microsecond)

print(f"""X_train shape: {X_train.shape}
y_train shape: {y_train.shape}
X_test shape: {X_test.shape}
y_test shape: {y_test.shape}""")

In [ ]:
class ECGDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = np.array(X).astype(np.float32)
        self.y = np.array(y).astype(np.float32)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]

        return X, y

    def __len__(self):
        return len(self.X)
    
train_dataset = ECGDataset(X_train, y_train)
test_dataset = ECGDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model

In [ ]:
device = torch.device("mps")
EPOCH = 70
MODEL_SAVE_PATH = "./model/"

In [ ]:
class DenseModel(nn.Module):
    def __init__(self):
        super(DenseModel, self).__init__()

        self.fc1 = nn.Linear(X_train.shape[1], 4096)
        self.fc1_2 = nn.Linear(4096, 2048)
        self.fc1_1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.fc7 = nn.Linear(32, 16)
        self.fc8 = nn.Linear(16, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc1_2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc1_1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc7(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc8(x)

        return x

In [ ]:
model = DenseModel().to(device)

criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
print(model)

# Model Train

In [ ]:
model.train()

train_loss, val_loss = [], []
for epoch in range(EPOCH):
    running_loss = 0.0
    for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch: {epoch + 1} / {EPOCH}"):
        inputs, labels = data
        inputs = inputs.reshape(-1, 1, 7500).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss.append(running_loss / len(train_dataloader))

    model.eval()
    with torch.no_grad():
        running_loss = 0.0
        for i, data in enumerate(test_dataloader):
            inputs, labels = data
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

        val_loss.append(running_loss / len(test_dataloader))
    
    print(f"Epoch: {epoch + 1} / {EPOCH} | Train loss: {train_loss[-1]:.5f} | Val loss: {val_loss[-1]:.5f}")
    
    # Model save
    if val_loss[-1] == min(val_loss):
        torch.save(model.state_dict(), MODEL_SAVE_PATH + f"model_{epoch + 1}_{val_loss[-1]:.5f}.pth")
        print(f"Model saved at Epoch: {epoch + 1}, Val loss: {val_loss[-1]:.5f}")

    model.train()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Train loss")
plt.plot(val_loss, label="Val loss")
plt.legend()
plt.show()

In [ ]:
model.eval()

output_list, label_list = [], []

with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        inputs = inputs.unsqueeze(1).to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        output_list.append(outputs.item())
        label_list.append(labels.item())

        print(f"Loss: {loss:.5f}\t\tPredicted: {outputs.item()}\t\tActual: {labels.item():.5f}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(output_list, label="Predicted", marker="o", linestyle="None", color="red")
plt.plot(label_list, label="Actual", marker="*", linestyle="None", color="blue")
plt.legend()
plt.show()